In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random

In [2]:
from tensorflow.keras.layers import Dense, Activation, Conv2D,Conv1D, Flatten,MaxPooling2D,BatchNormalization,Lambda, AveragePooling2D, MaxPooling1D
import tensorflow.keras.backend as K
from tensorflow.keras.layers import LSTM,Input,Bidirectional,Dense, Conv1D,MaxPooling1D,Flatten,Dropout,Input
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow import keras

In [3]:
df_X = pd.read_csv('dataset/train_features.csv')
df_y = pd.read_csv('dataset/train_target.csv')

In [4]:
df_X

,id,Time,S1,S2,S3,S4
0,0,0.000000,0.00,0.0,0.00,0.00
1,0,0.000004,0.00,0.0,0.00,0.00
2,0,0.000008,0.00,0.0,0.00,0.00
3,0,0.000012,0.00,0.0,0.00,0.00
4,0,0.000016,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...
1049995,2799,0.001480,-89256.32,-164614.6,48186.29,-147910.20
1049996,2799,0.001484,-29233.44,-179404.2,109534.40,-111591.10
1049997,2799,0.001488,37147.25,-190011.7,178781.10,-86836.78
1049998,2799,0.001492,98278.50,-190771.4,245726.70,-75660.00


In [120]:
def data_processing(data_frame,feature_list,conc_list=list()):
    for feature in feature_list:
        conc_list.append(np.asarray(data_frame[feature]).reshape(-1,375,1))
    conc_array = np.concatenate(conc_list,axis=2)
    array = conc_array.transpose(0,1,2)
    return array

In [126]:
data = data_processing(df_X,['S1','S2','S3','S4'])

In [127]:
y = df_y.drop(['id','V','M'],axis=1)
y_numpy = y.to_numpy()
y_list = y_numpy.tolist()

In [128]:
x_array = data
y_array = y_list
train_test_index = int(len(x_array)*0.8)
zip_list = list(zip(x_array,y_array))
random.shuffle(zip_list)
x_array,y_array = zip(*zip_list)
x_train,y_train = np.array(x_array[0:train_test_index]),np.array(y_array[0:train_test_index])
x_test,y_test = np.array(x_array[train_test_index:]),np.array(y_array[train_test_index:])

In [129]:
from sklearn.preprocessing import MinMaxScaler
Scaler = MinMaxScaler(feature_range=(-1, 1))
y_train_scaled = Scaler.fit_transform(y_train)
y_test_scaled = Scaler.fit_transform(y_test)
y_train_scaled.shape

(2240, 2)

In [144]:
model= Sequential(
    [
        Input(shape=(x_train.shape[1],x_train.shape[2])),
        # Use a Rescaling layer to make sure input values are in the [0, 1] range.
        # The original images have shape (28, 28), so we reshape them to (28, 28, 1)
        # Follow-up with a classic small convnet
        Conv1D(32, 2, activation="relu"),
        BatchNormalization(),
        MaxPooling1D(3),
        Conv1D(32,2, activation="relu"),
        MaxPooling1D(2),
        Conv1D(32,2, activation="relu"),
        Flatten(),
        Dense(64, activation="relu"),
        Dense(16, activation="relu"),
        Dense(2, activation='tanh')
    ]
)

In [145]:
def my_loss(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred))/2e+04

In [146]:
model.compile(loss=my_loss,optimizer='adam',metrics=['mean_absolute_error'])

In [147]:
from keras import callbacks
early_stopping = callbacks.EarlyStopping(monitor ="val_mean_absolute_error", 
                                        mode ="min", patience = 50, 
                                        restore_best_weights = True)

In [148]:
model.fit(x_train,y_train_scaled,validation_split=0.2,epochs=500,callbacks=[early_stopping])

Epoch 1/500
56/56 [==============================] - 2s 16ms/step - loss: 1.9707e-05 - mean_absolute_error: 0.5068 - val_loss: 6.4185e-06 - val_mean_absolute_error: 0.2781
Epoch 2/500
56/56 [==============================] - 1s 14ms/step - loss: 5.8438e-06 - mean_absolute_error: 0.2602 - val_loss: 3.6416e-06 - val_mean_absolute_error: 0.2056
Epoch 3/500
56/56 [==============================] - 1s 15ms/step - loss: 2.7825e-06 - mean_absolute_error: 0.1787 - val_loss: 2.4938e-06 - val_mean_absolute_error: 0.1684
Epoch 4/500
56/56 [==============================] - 1s 23ms/step - loss: 1.9714e-06 - mean_absolute_error: 0.1523 - val_loss: 1.6570e-06 - val_mean_absolute_error: 0.1351
Epoch 5/500
56/56 [==============================] - 1s 17ms/step - loss: 1.3314e-06 - mean_absolute_error: 0.1247 - val_loss: 1.2188e-06 - val_mean_absolute_error: 0.1138
Epoch 6/500
56/56 [==============================] - 1s 12ms/step - loss: 7.6593e-07 - mean_absolute_error: 0.0946 - val_loss: 1.3766e-06 - 

56/56 [==============================] - 1s 16ms/step - loss: 9.3899e-08 - mean_absolute_error: 0.0325 - val_loss: 3.1279e-07 - val_mean_absolute_error: 0.0499
Epoch 49/500
56/56 [==============================] - 1s 15ms/step - loss: 9.2833e-08 - mean_absolute_error: 0.0325 - val_loss: 2.4936e-07 - val_mean_absolute_error: 0.0454
Epoch 50/500
56/56 [==============================] - 1s 14ms/step - loss: 9.1646e-08 - mean_absolute_error: 0.0319 - val_loss: 2.4648e-07 - val_mean_absolute_error: 0.0434
Epoch 51/500
56/56 [==============================] - 1s 14ms/step - loss: 7.3566e-08 - mean_absolute_error: 0.0291 - val_loss: 2.4144e-07 - val_mean_absolute_error: 0.0434
Epoch 52/500
56/56 [==============================] - 1s 14ms/step - loss: 9.0324e-08 - mean_absolute_error: 0.0314 - val_loss: 2.2552e-07 - val_mean_absolute_error: 0.0427
Epoch 53/500
56/56 [==============================] - 1s 15ms/step - loss: 7.5488e-08 - mean_absolute_error: 0.0292 - val_loss: 2.2315e-07 - val_mea

56/56 [==============================] - 1s 15ms/step - loss: 4.0896e-08 - mean_absolute_error: 0.0214 - val_loss: 1.4377e-07 - val_mean_absolute_error: 0.0331
Epoch 143/500
56/56 [==============================] - 1s 15ms/step - loss: 2.9501e-08 - mean_absolute_error: 0.0183 - val_loss: 1.3160e-07 - val_mean_absolute_error: 0.0302
Epoch 144/500
56/56 [==============================] - 1s 15ms/step - loss: 2.7689e-08 - mean_absolute_error: 0.0176 - val_loss: 1.2729e-07 - val_mean_absolute_error: 0.0304
Epoch 145/500
56/56 [==============================] - 1s 14ms/step - loss: 2.5505e-08 - mean_absolute_error: 0.0169 - val_loss: 1.2398e-07 - val_mean_absolute_error: 0.0301
Epoch 146/500
56/56 [==============================] - 1s 15ms/step - loss: 2.9176e-08 - mean_absolute_error: 0.0181 - val_loss: 1.2049e-07 - val_mean_absolute_error: 0.0298
Epoch 147/500
56/56 [==============================] - 1s 14ms/step - loss: 2.6537e-08 - mean_absolute_error: 0.0174 - val_loss: 1.2702e-07 - va

56/56 [==============================] - 1s 14ms/step - loss: 1.5451e-08 - mean_absolute_error: 0.0133 - val_loss: 1.0516e-07 - val_mean_absolute_error: 0.0275
Epoch 237/500
56/56 [==============================] - 1s 14ms/step - loss: 1.7145e-08 - mean_absolute_error: 0.0137 - val_loss: 9.7841e-08 - val_mean_absolute_error: 0.0266
Epoch 238/500
56/56 [==============================] - 1s 14ms/step - loss: 1.7496e-08 - mean_absolute_error: 0.0139 - val_loss: 9.9811e-08 - val_mean_absolute_error: 0.0267
Epoch 239/500
56/56 [==============================] - 1s 14ms/step - loss: 1.5224e-08 - mean_absolute_error: 0.0132 - val_loss: 9.7177e-08 - val_mean_absolute_error: 0.0257
Epoch 240/500
56/56 [==============================] - 1s 14ms/step - loss: 1.6607e-08 - mean_absolute_error: 0.0136 - val_loss: 9.8752e-08 - val_mean_absolute_error: 0.0259
Epoch 241/500
56/56 [==============================] - 1s 14ms/step - loss: 1.5645e-08 - mean_absolute_error: 0.0134 - val_loss: 1.0654e-07 - va

56/56 [==============================] - 1s 15ms/step - loss: 1.1799e-08 - mean_absolute_error: 0.0117 - val_loss: 9.4389e-08 - val_mean_absolute_error: 0.0254
Epoch 331/500
56/56 [==============================] - 1s 15ms/step - loss: 1.3253e-08 - mean_absolute_error: 0.0119 - val_loss: 9.0296e-08 - val_mean_absolute_error: 0.0246
Epoch 332/500
56/56 [==============================] - 1s 14ms/step - loss: 1.1450e-08 - mean_absolute_error: 0.0113 - val_loss: 8.9671e-08 - val_mean_absolute_error: 0.0242
Epoch 333/500
56/56 [==============================] - 1s 15ms/step - loss: 1.2099e-08 - mean_absolute_error: 0.0116 - val_loss: 9.1378e-08 - val_mean_absolute_error: 0.0248
Epoch 334/500
56/56 [==============================] - 1s 14ms/step - loss: 1.2956e-08 - mean_absolute_error: 0.0119 - val_loss: 9.1851e-08 - val_mean_absolute_error: 0.0249
Epoch 335/500
56/56 [==============================] - 1s 15ms/step - loss: 1.1773e-08 - mean_absolute_error: 0.0115 - val_loss: 9.1712e-08 - va

Epoch 424/500
56/56 [==============================] - 1s 14ms/step - loss: 1.0196e-08 - mean_absolute_error: 0.0102 - val_loss: 8.0315e-08 - val_mean_absolute_error: 0.0232
Epoch 425/500
56/56 [==============================] - 1s 14ms/step - loss: 1.0439e-08 - mean_absolute_error: 0.0108 - val_loss: 8.6273e-08 - val_mean_absolute_error: 0.0235
Epoch 426/500
56/56 [==============================] - 1s 14ms/step - loss: 1.0355e-08 - mean_absolute_error: 0.0106 - val_loss: 8.2082e-08 - val_mean_absolute_error: 0.0234
Epoch 427/500
56/56 [==============================] - 1s 15ms/step - loss: 1.0036e-08 - mean_absolute_error: 0.0105 - val_loss: 9.0066e-08 - val_mean_absolute_error: 0.0248
Epoch 428/500
56/56 [==============================] - 1s 15ms/step - loss: 1.0729e-08 - mean_absolute_error: 0.0109 - val_loss: 8.1641e-08 - val_mean_absolute_error: 0.0233
Epoch 429/500
56/56 [==============================] - 1s 14ms/step - loss: 1.0180e-08 - mean_absolute_error: 0.0107 - val_loss: 8

# -400 to 400

In [149]:
model.evaluate(x_test,y_test_scaled)

18/18 [==============================] - 0s 4ms/step - loss: 8.0875e-08 - mean_absolute_error: 0.0219


[8.087486946806166e-08, 0.021893542259931564]

In [155]:
model.save('model/model_xny.h5')